In [ ]:
!kaggle datasets download -d alistairking/recyclable-and-household-waste-classification

Dataset URL: https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification
License(s): MIT
 99% 915M/920M [00:09<00:00, 89.9MB/s]
100% 920M/920M [00:09<00:00, 102MB/s] 


In [ ]:
!unzip \*.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/images/plastic_shopping_bags/default/Image_1.png  
  inflating: images/images/plastic_shopping_bags/default/Image_10.png  
  inflating: images/images/plastic_shopping_bags/default/Image_100.png  
  inflating: images/images/plastic_shopping_bags/default/Image_101.png  
  inflating: images/images/plastic_shopping_bags/default/Image_102.png  
  inflating: images/images/plastic_shopping_bags/default/Image_103.png  
  inflating: images/images/plastic_shopping_bags/default/Image_104.png  
  inflating: images/images/plastic_shopping_bags/default/Image_105.png  
  inflating: images/images/plastic_shopping_bags/default/Image_106.png  
  inflating: images/images/plastic_shopping_bags/default/Image_107.png  
  inflating: images/images/plastic_shopping_bags/default/Image_108.png  
  inflating: images/images/plastic_shopping_bags/default/Image_109.png  
  inflating: images/images/plastic_shopping_bags/default/Image_11.png  
  in

In [ ]:
!pip install torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class WasteDataset(Dataset):
    def __init__(self, data, transform=None):
        """
        Args:
            data (list): List of tuples (image_path, label)
            transform (callable, optional): Optional transform to be applied on a sample.
      """
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        try:
            image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Return a black image in case of error
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:

import os
from sklearn.model_selection import train_test_split
from torchvision import transforms # Import the transforms module from torchvision
from torchvision.datasets import ImageFolder

# Define the mapping of waste categories to the 4 main classes
waste_class_mapping = {
    'aerosol_cans': 'hazardous',
    'aluminum_food_cans': 'non_biodegradable',
    'aluminum_soda_cans': 'non_biodegradable',
    'cardboard_boxes': 'biodegradable',
    'cardboard_packaging': 'biodegradable',
    'clothing': 'non_biodegradable',
    'coffee_grounds': 'biodegradable',
    'disposable_plastic_cutlery': 'non_biodegradable',
    'eggshells': 'biodegradable',
    'food_waste': 'biodegradable',
    'glass_beverage_bottles': 'non_biodegradable',
    'glass_cosmetic_containers': 'non_biodegradable',
    'glass_food_jars': 'non_biodegradable',
    'magazines': 'biodegradable',
    'newspaper': 'biodegradable',
    'office_paper': 'biodegradable',
    'paper_cups': 'biodegradable',
    'plastic_cup_lids': 'non_biodegradable',
    'plastic_detergent_bottles': 'non_biodegradable',
    'plastic_food_containers': 'non_biodegradable',
    'plastic_shopping_bags': 'non_biodegradable',
    'plastic_soda_bottles': 'non_biodegradable',
    'plastic_straws': 'non_biodegradable',
    'plastic_trash_bags': 'non_biodegradable',
    'plastic_water_bottles': 'non_biodegradable',
    'shoes': 'non_biodegradable',
    'steel_food_cans': 'non_biodegradable',
    'styrofoam_cups': 'non_biodegradable',
    'styrofoam_food_containers': 'non_biodegradable',
    'tea_bags': 'biodegradable'
}

# Define the four main classes
main_classes = ['biodegradable', 'non_biodegradable', 'trash', 'hazardous']

# Create a reverse mapping from class names to indices
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(main_classes)}

# Set dataset directory (update the path accordingly)
dataset_dir = '/content/images/images'

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match VGG16 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Using ImageNet mean and std
                         [0.229, 0.224, 0.225])
])

# Load the original ImageFolder dataset
full_dataset = ImageFolder(dataset_dir, transform=None)  # Set transform=None for now

# Re-label the dataset based on the waste_class_mapping
def map_classes(image_folder_dataset, mapping, class_to_idx):
    mapped_dataset = []
    for path, label in image_folder_dataset.samples:
        original_class_name = image_folder_dataset.classes[label]
        new_class = mapping.get(original_class_name)
        if new_class:
            mapped_label = class_to_idx[new_class]  # New class index
            mapped_dataset.append((path, mapped_label))
    return mapped_dataset

mapped_dataset = map_classes(full_dataset, waste_class_mapping, class_to_idx)

# Handle the 'trash' class if needed
# If any of your categories should map to 'trash', include them in the mapping above

# Split into train/test set (80% train, 20% test)
train_data, test_data = train_test_split(mapped_dataset, test_size=0.2, random_state=42, stratify=[label for _, label in mapped_dataset])


In [ ]:
import torch
from torch.utils.data import DataLoader

# Ensure WasteDataset is correctly implemented
train_dataset = WasteDataset(train_data, transform=transform)
test_dataset = WasteDataset(test_data, transform=transform)

# Define batch size
batch_size = 32

# Use optimized DataLoader settings
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,  # Increase based on system capabilities
    pin_memory=torch.cuda.is_available(),
    persistent_workers=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,  # Increase for better performance
    pin_memory=torch.cuda.is_available(),
    persistent_workers=True if torch.cuda.is_available() else False
)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Check a single batch
data_iter = iter(train_loader)
images, labels = next(data_iter)

print(f'Images shape: {images.shape}')  # Should be [batch_size, 3, 224, 224]
print(f'Labels shape: {labels.shape}')  # Should be [batch_size]
print(f'Labels: {labels}')  # Check if labels are within [0, 3]

Images shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Labels: tensor([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 0, 0, 1, 1, 1, 1])


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
import numpy as np

# Load VGG16 model with pre-trained weights
model = models.vgg16(pretrained=True)

# Freeze most layers, unfreeze last 4 conv layers
for param in model.features[:-4].parameters():
    param.requires_grad = False

# Modify classifier to match 4 output classes
model.classifier[6] = nn.Linear(4096, 4)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

# Define Early Stopping criteria
early_stopping_patience = 5

def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs):
    best_loss = np.inf
    patience_counter = 0

    train_loss_history = []
    test_loss_history = []
    train_acc_history = []
    test_acc_history = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        train_loss_history.append(epoch_loss)
        train_acc_history.append(epoch_acc)

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(test_loader.dataset)
        val_acc = val_correct / val_total
        test_loss_history.append(val_loss)
        test_acc_history.append(val_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Reduce LR on plateau
        scheduler.step(val_loss)
    return train_loss_history, test_loss_history, train_acc_history, test_acc_history

# Train the model
num_epochs = 40
train_loss_history, test_loss_history, train_acc_history, test_acc_history = train_model(
    model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs
)

Epoch [1/40], Train Loss: 0.4284, Train Acc: 0.8697, Val Loss: 0.3454, Val Acc: 0.8683
Epoch [2/40], Train Loss: 0.3097, Train Acc: 0.9158, Val Loss: 0.2736, Val Acc: 0.9263
Epoch [3/40], Train Loss: 0.1974, Train Acc: 0.9428, Val Loss: 0.2216, Val Acc: 0.9437
Epoch [4/40], Train Loss: 0.2345, Train Acc: 0.9493, Val Loss: 0.3399, Val Acc: 0.9300
Epoch [5/40], Train Loss: 0.1864, Train Acc: 0.9544, Val Loss: 0.2739, Val Acc: 0.9403
Epoch [6/40], Train Loss: 0.1611, Train Acc: 0.9710, Val Loss: 0.3135, Val Acc: 0.9450
Epoch [7/40], Train Loss: 0.0562, Train Acc: 0.9875, Val Loss: 0.5162, Val Acc: 0.9433
Epoch [8/40], Train Loss: 0.0253, Train Acc: 0.9928, Val Loss: 0.3898, Val Acc: 0.9497
Epoch [9/40], Train Loss: 0.0177, Train Acc: 0.9958, Val Loss: 0.5466, Val Acc: 0.9563
Epoch [10/40], Train Loss: 0.0133, Train Acc: 0.9966, Val Loss: 0.4899, Val Acc: 0.9480
Epoch [11/40], Train Loss: 0.0076, Train Acc: 0.9975, Val Loss: 0.6065, Val Acc: 0.9527
Epoch [12/40], Train Loss: 0.0050, Train 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

# Load the best model weights
model.load_state_dict(torch.load('best_model.pt'))
model.to(device)
model.eval()

# Real-time example testing
def test_real_time_image(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')

    # Display the image
    plt.imshow(image)
    plt.axis('off')  # Turn off axis labels
    plt.show()

    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    image_tensor = transform_test(image).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        _, preds = torch.max(outputs, 1)

    # Map index to class name
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    predicted_class = idx_to_class[preds.item()]

    print(f'Predicted Class: {predicted_class}')

# Example usage
# Replace 'path_to_real_image.jpg' with the path to your test image
test_image_path = '/content/test.1.jfif'
test_real_time_image(test_image_path, model)


<ipython-input-9-cba3f8ff9ad3>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pt'))


FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pt'

In [ ]:
import torch

# Save the trained model's state_dict
torch.save(model.state_dict(), 'image_classification_model.pth')

# For Google Colab: Download the model to your local system
from google.colab import files
files.download('image_classification_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>